### This notebook will illustrate on how to access Wikidata using SPARQL

### Import SPARQLWrapper and Pandas

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

#### The function will fetch all the values for which Mumbai is an instance of and converts the result into json and further into dataframe

In [2]:
def access_wiki_data():
    '''
    Function to make a call to wikidata. set the wikidata sparql endpoint and query.
    wd:Q1156 is an id for the entity "Mumbai". P31 is an id for 'instanceof' property
    '''
    sparql = SPARQLWrapper("http://query.wikidata.org/sparql")
    sparql.setQuery("""
                    PREFIX wd: <http://www.wikidata.org/entity/>
                    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                    
                    SELECT ?item
                    WHERE 
                    {
                        wd:Q1156 wdt:P31 ?item .
                    }  
                  """)
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    results_df = pd.io.json.json_normalize(result['results']['bindings'])
    return results_df

In [3]:
results = access_wiki_data()
results

,item.type,item.value
0,uri,http://www.wikidata.org/entity/Q515


#### The below function will show how to access the labels. 

In [4]:
def access_wiki_data_withLabel():
    sparql = SPARQLWrapper("http://query.wikidata.org/sparql")
    sparql.setQuery("""
                    PREFIX wd: <http://www.wikidata.org/entity/>
                    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                    PREFIX wikibase: <http://wikiba.se/ontology#>
                    PREFIX bd: <http://www.bigdata.com/rdf#>
                    
                    SELECT ?item ?itemLabel
                    WHERE 
                    {
                        wd:Q1156 wdt:P31 ?item .
                        SERVICE wikibase:label {bd:serviceParam wikibase:language "en" .}
                    } 
                    
                  """)
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    results_df = pd.io.json.json_normalize(result['results']['bindings'])
    return results_df

In [5]:
results = access_wiki_data_withLabel()
results

,item.type,item.value,itemLabel.type,itemLabel.value,itemLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q515,literal,city,en


#### The below function will show to how to access the data with entity name. for example with entity name as "Mumbai" instead of entity id for Mumbai which is Q1156

In [6]:
def access_wiki_data_with_entity_name():
    sparql = SPARQLWrapper("http://query.wikidata.org/sparql")
    sparql.setQuery("""
                    PREFIX wd: <http://www.wikidata.org/entity/>
                    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                    PREFIX wikibase: <http://wikiba.se/ontology#>
                    PREFIX bd: <http://www.bigdata.com/rdf#>
                    
                    SELECT ?itemLabel ?prop ?propLabel ?value ?valueLabel
                    WHERE 
                    {
                        ?item ?prop "Mumbai"@en .
                        ?item wdt:P31 ?value .
                        SERVICE wikibase:label {bd:serviceParam wikibase:language "en" .}
                    } 
                    
                  """)
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    results_df = pd.io.json.json_normalize(result['results']['bindings'])
    return results_df

In [7]:
results = access_wiki_data_with_entity_name()
results

,itemLabel.type,itemLabel.value,itemLabel.xml:lang,prop.type,prop.value,propLabel.type,propLabel.value,value.type,value.value,valueLabel.type,valueLabel.value,valueLabel.xml:lang
0,literal,Mumbai,en,uri,http://www.w3.org/2000/01/rdf-schema#label,literal,http://www.w3.org/2000/01/rdf-schema#label,uri,http://www.wikidata.org/entity/Q515,literal,city,en
1,literal,Mumbai,en,uri,http://www.w3.org/2000/01/rdf-schema#label,literal,http://www.w3.org/2000/01/rdf-schema#label,uri,http://www.wikidata.org/entity/Q4167410,literal,Wikimedia disambiguation page,en


In [8]:
results[['itemLabel.value', 'value.value', 'valueLabel.value']]

,itemLabel.value,value.value,valueLabel.value
0,Mumbai,http://www.wikidata.org/entity/Q515,city
1,Mumbai,http://www.wikidata.org/entity/Q4167410,Wikimedia disambiguation page


#### This function will show to parameterize the SPARQL Query. This function will take name as input and set it to SPARQL query

In [9]:
def access_wiki_with_parameterized_data(name):
    name = '"'+name+'"'
    sparql = SPARQLWrapper("http://query.wikidata.org/sparql")
    sparql.setQuery("""
                    PREFIX wd: <http://www.wikidata.org/entity/>
                    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                    PREFIX wikibase: <http://wikiba.se/ontology#>
                    PREFIX bd: <http://www.bigdata.com/rdf#>
                    
                    SELECT ?itemLabel ?prop ?propLabel ?value ?valueLabel
                    WHERE 
                    {
                        ?item ?prop %s@en .
                        ?item wdt:P31 ?value .
                        SERVICE wikibase:label {bd:serviceParam wikibase:language "en" .}
                    } 
                    
                  """ % name)
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    results_df = pd.io.json.json_normalize(result['results']['bindings'])
    return results_df

In [10]:
results = access_wiki_with_parameterized_data("Mumbai")
results

,itemLabel.type,itemLabel.value,itemLabel.xml:lang,prop.type,prop.value,propLabel.type,propLabel.value,value.type,value.value,valueLabel.type,valueLabel.value,valueLabel.xml:lang
0,literal,Mumbai,en,uri,http://www.w3.org/2000/01/rdf-schema#label,literal,http://www.w3.org/2000/01/rdf-schema#label,uri,http://www.wikidata.org/entity/Q515,literal,city,en
1,literal,Mumbai,en,uri,http://www.w3.org/2000/01/rdf-schema#label,literal,http://www.w3.org/2000/01/rdf-schema#label,uri,http://www.wikidata.org/entity/Q4167410,literal,Wikimedia disambiguation page,en


In [11]:
results[['itemLabel.value', 'value.value', 'valueLabel.value']]

,itemLabel.value,value.value,valueLabel.value
0,Mumbai,http://www.wikidata.org/entity/Q515,city
1,Mumbai,http://www.wikidata.org/entity/Q4167410,Wikimedia disambiguation page
